# Lab 9 - DEFI(Uniswap V2)

## 1. Setup

### Step 1: Init NPM project

In [ ]:
mkdir test
mkdir contracts
mkdir build
npm init -y

### Step 2. Install Hardhat Packages

In [ ]:
npm i -D hardhat@2.15.0 @nomiclabs/hardhat-ethers@2.2.3 @nomiclabs/hardhat-etherscan@3.1.7 hardhat-ignore-warnings chai@4.3.7 @openzeppelin/contracts@3.4.2
npm i dotenv@16.1.4 ethers@5.7.2



### Step 3. Install UniswapV2 Packages

In [ ]:
npm i @uniswap/lib@4.0.1-alpha @uniswap/sdk@3.0.3 @uniswap/v2-core@1.0.1

### Step 4 - Download UniswapV2 contract source code

a) Clone the uniswap repositories into `\tmp\v2-core` and `\tmp\v2-periphery`.
b) Checkout the commits used for mainnet deployment
c) Copy `\tmp\v2-core\contracts` and `\tmp\v2-periphery\contracts` directory into `/lab-9/contracts`.

In [ ]:
mkdir /tmp/uniswap
bash -c "git clone https://github.com/Uniswap/v2-periphery /tmp/uniswap/v2-periphery && cd /tmp/uniswap/v2-periphery && git checkout 69617118cda519dab608898d62aaa79877a61004"
cp -rp /tmp/uniswap/v2-periphery/contracts contracts/v2-periphery
bash -c "git clone https://github.com/Uniswap/v2-core /tmp/uniswap/v2-core && cd /tmp/uniswap/v2-core && git checkout 816075049f811f1b061bca81d5d040b96f4c07eb"
cp -rp /tmp/uniswap/v2-core/contracts contracts/v2-core

### Step 5. Create Hardhat Configuration

* Create hardhat.config.js
* Use compiler version 0.5.16 for compiling v2-core.
* Use compiler version 0.6.6 for compiling v2-periphery. Optimizer must be enabled otherwise the Router bytecode will be too large to deploy.

```js
require("@nomiclabs/hardhat-ethers");
module.exports = {
    solidity: {
        compilers: [
            {
                version: "0.5.16",
            },
            {
                version: "0.6.6",
                settings: {
                    optimizer: {
                        enabled: true,
                        runs: 200,
                    },
                },
            },
        ],
    },
};
```

The completed project structure should look like the following:

```
/lab-9
    /contracts
        /v2-core
            /interfaces
            /libraries
            UniswapV2ERC20.sol
            UniswapV2Factory.sol
            UniswapV2Pair.sol
        /v2-periphery
            /interfaces
            /libraries
            UniswapV2Router02.sol
```


### Step 6. Compile

This should compile the Uniswap contracts into the `artifacts` directory.

In [ ]:
hh compile

### Step 7. Generate init code hash for UniswapV2Pair

The function **pairFor()** in the **UniswapV2Library** uses an offchain technique to get the pool contract's address by hardcoding the bytecode's hash into the contract. Because the bytecode can be different when compiled on your machine, from the one used in the contract, this function has to replace the hashcode using the instructions below.

**a) Launch hardhat console.**

```sh
hh console
```

**b) Enter the following command from within Hardhat console**

```
> json = require("./artifacts/contracts/v2-core/UniswapV2Pair.sol/UniswapV2Pair.json");
> console.log(ethers.utils.keccak256(json.bytecode));
```
The output should show: `215a032792ab9f4a5eb14f1f4c1daed5017b1eee4de72ddb42e06c967b16c5d4`

**c) Open the file `v2-periphery/libraries/UniswapV2Library.sol`.**

**d) Replace line 24 of  with the hash.**

```js
function pairFor(address factory, address tokenA, address tokenB) internal pure returns (address pair) {
    (address token0, address token1) = sortTokens(tokenA, tokenB);
    pair = address(uint(keccak256(abi.encodePacked(
            hex'ff',
            factory,
            keccak256(abi.encodePacked(token0, token1)),
            hex'215a032792ab9f4a5eb14f1f4c1daed5017b1eee4de72ddb42e06c967b16c5d4'
            //hex'96e8ac4277198ff8b6f785478aa9a39f403cb768dd02cbee326c3e7da348845f' // init code hash
        ))));
}
```

**e) Compile again.**

hh compile